## Feature Engineering_Momentum

## Features

- RSI
- Stochastic RSI
- Awesome oscillator
- Accelerator oscillator

__Load data__

__Data Source:__ lob_sample_data.parquet

In [2]:
import pandas as pd

df = pd.read_parquet('lob_sample_data.parquet', engine='pyarrow')

In [3]:
df.head()

,Timestamp,Exchange,Bid,Ask,Date,Mid_Price
0,0.000,Exch0,[],[],2025-01-02,NaN
1,0.279,Exch0,"[[1, 6]]",[],2025-01-02,NaN
2,1.333,Exch0,"[[1, 6]]","[[800, 1]]",2025-01-02,400.5
3,1.581,Exch0,"[[1, 6]]","[[799, 1]]",2025-01-02,400.0
4,1.643,Exch0,"[[1, 6]]","[[798, 1]]",2025-01-02,399.5


In [4]:
import ast

#convert string to lists
df['Bid'] = df['Bid'].apply(ast.literal_eval)
df['Ask'] = df['Ask'].apply(ast.literal_eval)

In [5]:
#drop missing rows in mid price
df = df.dropna(subset=['Mid_Price'])

__RSI__

RSI (relative strength index)-popular- overbought and oversold conditions-calculates with avg gain, avg loss and relative strength. You need to check direction not just value to understand trend. 

- above 70- overbought
- below 30- oversold
- 50-70- a strong bullish trend
- 30-50 -a moderate bullish trend
- 50-30- a strong bearish trend
- 70-40- a moderate bearish trend

In [6]:
#rsi
delta = df['Mid_Price'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=20).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=20).mean()
RS = gain / loss

df['RSI'] = 100 - (100 / (1 + RS))

__Stochastic RSI__

The Stochastic RSI is an indicator used to identify overbought and oversold conditions by measuring the level of the RSI relative to its high and low range over a specific period. It is derived from the RSI but provides more sensitivity and a higher frequency of trading signals by applying the stochastic oscillator formula to RSI values.

$$ \text{StochRSI} = \frac{\text{RSI} - \text{Min(RSI, n)}}{\text{Max(RSI, n)} - \text{Min(RSI, n)}} $$

where:
- $\text{RSI}$ is the current value of the Relative Strength Index,
- $\text{Min(RSI, n)}$ is the minimum RSI value over the last $n$ periods,
- $\text{Max(RSI, n)}$ is the maximum RSI value over the last $n$ periods,
- $n$ is the specified period over which the highs and lows are calculated.

In [7]:
#min and max RSI values over window
min_rsi = df['RSI'].rolling(window=20).min()
max_rsi = df['RSI'].rolling(window=20).max()

#calc Stochastic RSI
df['Stochastic_RSI'] = (df['RSI'] - min_rsi) / (max_rsi - min_rsi)

In [10]:
#scale to 0-100 range (better for interpretation and comparison to other oscillators)
df['Stochastic_RSI'] = df['Stochastic_RSI'] * 100

__Awesome oscillator__

The awesome oscillator is a market momentum indicator that compares the recent market momentum, with the momentum over a broader timeframe, by calculating the difference between a 34 period and a 5 period simple moving averages (SMA) of the median prices (or mid prices when median prices are not available).
Calc by-

$$
AO = SMA_{5} - SMA_{34}
$$

where
- $SMA_{5}$ is the 5-period simple moving average of the mid-prices.
- $SMA_{34}$ is the 34-period simple moving average of the mid-prices.

The AO serves to detect changes in the market's momentum and potentially signal upcoming market reversals.

In [11]:
#calculate sma for 34 and 5 periods
sma_34 = df['Mid_Price'].rolling(window=34, min_periods=1).mean()
sma_5 = df['Mid_Price'].rolling(window=5, min_periods=1).mean()

#calc ao
df['Awesome_Oscillator'] = sma_5 - sma_34

__Accelerator oscillator__

The accelerator oscillator measures the acceleration or deceleration of the current market driving force, essentially indicating whether the market force is increasing or decreasing.

Its calculated as the difference between the 5-period simple moving average of the awesome oscillator, and the ao itself:

$$
AC = AO - SMA_{5}(AO)
$$

where-
- $AO$ is the awesome oscillator.
- $SMA_{5}(AO)$ is the 5-period simple moving average of the awesome oscillator values.

It can be used to confirm ao signals or predict possible reversals.

In [12]:
#calc acc oscilator
ao_sma_5 = df['Awesome_Oscillator'].rolling(window=5, min_periods=1).mean()
df['Accelerator_Oscillator'] = df['Awesome_Oscillator'] - ao_sma_5